In [1]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import torch.nn.functional as F
from skopt import BayesSearchCV
from skopt.space import Real
from skopt import gp_minimize
import torch.nn as nn
import seaborn as sns
from scipy.optimize import minimize
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib as mpl
from skopt.utils import use_named_args
from tqdm import tqdm
import multiprocessing
import threading

In [2]:
# 设置GPU设备，如果没有可用的GPU，则使用CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(888888)

print(device)

cpu


In [3]:
# 基础参数
epochs = 400000    # 训练代数
h = 100    # 画图网格密度
N = 100    # 内点配置点数
N1 = 0.01    # 边界点配置点数
n = 1000    # PDE数据点
top_k = 100  # 前n个残差最大的点
num_new_points = 10  # 以圆心生成的n个点
bias = 0.001   # 圆半径

In [4]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# Loss
loss = torch.nn.MSELoss()
# loss = torch.nn.L1Loss()  #MAE
# loss = torch.nn.SmoothL1Loss(beta=1.0)

# 递归求导
def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True,
                                   only_inputs=True, )[0]
    else:
        return gradients(gradients(u, x), x, order=order - 1)

# 区域1（-1, +delta）

In [15]:
def point_1(n=100, device=device):
    x = torch.rand(n, 1, device=device, requires_grad=True) * 1.05 - 1
    y = torch.rand(n, 1, device=device, requires_grad=True) * 2 - 1
    return x, y


def l_ugeq_1(u, n=100):
    # u >= 0
    x, y = point_1(n, device)
    cond = torch.zeros_like(y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    llos = torch.relu(-uxy)
    ugeq_loss = 100 * torch.sum(llos)
    
    # 返回每个点的损失值
    return ugeq_loss, llos, x, y

def l_pde_1(u, n=100, device=device):
    # 等式项损失
    x, y = point_1(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    
    # 确保梯度计算返回每个点的值
    grad_x2 = gradients(uxy, x, 2)  # 对 x 计算二阶梯度
    grad_y2 = gradients(uxy, y, 2)  # 对 y 计算二阶梯度

    # 计算每个点的损失
    loss_value = -grad_x2 - grad_y2  # 假设这里是 PDE 中的计算
    loss_value = loss_value - cond   # 计算 PDE 的残差
    pde_loss = loss(-gradients(uxy, x, 2) - gradients(uxy, y, 2)- cond, torch.zeros_like(cond))

    # 返回每个点的损失值
    return pde_loss, loss_value, x, y

def l_JU_1(u, n=100, device=device):
    """
    计算每个点的 JU 损失值和整体的 JU 损失（通过蒙特卡罗积分近似）
    
    参数：
    - u: 神经网络
    - n: 采样点的数量
    - device: 设备（"cuda" 或 "cpu"）
    
    返回：
    - JU_loss: 总的目标函数值（通过积分近似）
    - llos_JU: 每个点的损失值（对应每个采样点）
    - x, y: 对应每个点的坐标
    """
    # 随机采样点
    x, y = point_1(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

    # 计算 u 和其梯度
    uxy = u(torch.cat([x, y], dim=1))
    grad_x = gradients(uxy, x, 1)
    grad_y = gradients(uxy, y, 1)
    
    # 每个点的损失值
    llos_JU = 0.5 * (grad_x**2 + grad_y**2) - cond * uxy  # 每个点对应的损失值
    p1 = 0.5 * (grad_x**2 + grad_y**2)
    p2 = cond * uxy

    # 使用蒙特卡罗积分近似整体损失
    JU_loss = llos_JU.mean()  # 取所有点的平均值作为积分的近似

    return JU_loss, llos_JU, x, y, p1, p2



# 区域2（-delta, 1）

In [7]:
def point_2(n=100, device=device):
    x = torch.rand(n, 1, device=device, requires_grad=True) * 1.05 - 0.05
    y = torch.rand(n, 1, device=device, requires_grad=True) * 2 - 1
    return x, y


def l_ugeq_2(u, n=100):
    # u >= 0
    x, y = point_2(n, device)
    cond = torch.zeros_like(y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    llos = torch.relu(-uxy)
    ugeq_loss = 100 * torch.sum(llos)
    
    # 返回每个点的损失值
    return ugeq_loss, llos, x, y

def l_pde_2(u, n=100, device=device):
    # 等式项损失
    x, y = point_2(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    
    # 确保梯度计算返回每个点的值
    grad_x2 = gradients(uxy, x, 2)  # 对 x 计算二阶梯度
    grad_y2 = gradients(uxy, y, 2)  # 对 y 计算二阶梯度

    # 计算每个点的损失
    loss_value = -grad_x2 - grad_y2  # 假设这里是 PDE 中的计算
    loss_value = loss_value - cond   # 计算 PDE 的残差
    pde_loss = loss(-gradients(uxy, x, 2) - gradients(uxy, y, 2)- cond, torch.zeros_like(cond))

    # 返回每个点的损失值
    return pde_loss, loss_value, x, y

def l_JU_2(u, n=100, device=device):
    """
    计算每个点的 JU 损失值和整体的 JU 损失（通过蒙特卡罗积分近似）
    
    参数：
    - u: 神经网络
    - n: 采样点的数量
    - device: 设备（"cuda" 或 "cpu"）
    
    返回：
    - JU_loss: 总的目标函数值（通过积分近似）
    - llos_JU: 每个点的损失值（对应每个采样点）
    - x, y: 对应每个点的坐标
    """
    # 随机采样点
    x, y = point_2(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

    # 计算 u 和其梯度
    uxy = u(torch.cat([x, y], dim=1))
    grad_x = gradients(uxy, x, 1)
    grad_y = gradients(uxy, y, 1)
    
    # 每个点的损失值
    llos_JU = 0.5 * (grad_x**2 + grad_y**2) - cond * uxy  # 每个点对应的损失值
    p1 = 0.5 * (grad_x**2 + grad_y**2)
    p2 = cond * uxy

    # 使用蒙特卡罗积分近似整体损失
    JU_loss = llos_JU.mean()  # 取所有点的平均值作为积分的近似

    return JU_loss, llos_JU, x, y, p1, p2



In [8]:
# 边界函数构建
def l_boundary1(u):
    # 下边界
    x = torch.arange(-1.001, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = -torch.ones_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary2(u):
    # 上边界
    x = torch.arange(-1.001, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = torch.ones_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary3(u):
    # 左边界
    y = torch.arange(-1.001, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = -torch.ones_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary4(u):
    # 右边界
    y = torch.arange(-1.001, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = torch.ones_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)


def l_boundary5(u):
    # y轴
    y = torch.arange(-1.001, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = torch.zeros_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary6(u):
    # x轴
    x = torch.arange(-1.001, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = torch.zeros_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

# 查看情况代码

In [19]:
def record_loss_and_save_model(u, epoch, maxerror, current_time, h, device):
    """
    每100次迭代记录一次损失并保存模型。

    参数:
    - u: 模型
    - epoch: 当前训练的 epoch 数
    - maxerror: 当前最小的误差
    - current_time: 上一次记录的时间
    - h: 网格划分大小
    - device: 设备
    返回:
    - maxerror: 更新后的最小误差
    - current_time: 更新后的时间
    """
    # 生成网格点
    xc_x = torch.linspace(-1, 1, h, device=device)
    xc_y = torch.linspace(-1, 1, h, device=device)
    xm, ym = torch.meshgrid(xc_x, xc_y)
    xx = xm.reshape(-1, 1)
    yy = ym.reshape(-1, 1)
    xy = torch.cat([xx, yy], dim=1).to(device)

    # 计算预测值和真实值
    u_pred = torch.relu(u(xy))
    u_real = torch.relu(torch.sin(torch.pi * xx) * torch.sin(torch.pi * yy))
    u_error = torch.abs(u_pred - u_real)

    # 计算误差网格
    u_pred_fig = u_pred.reshape(h, h)
    u_real_fig = u_real.reshape(h, h)
    u_error_fig = u_error.reshape(h, h)

    # 计算当前最大绝对误差
    max_abs_error = float(torch.max(u_error))
    print(f"At epoch {epoch}, time_speed: {abs(current_time - time.time()):.2f}s, Max abs error is: {max_abs_error}, best: {maxerror}")
    print('-----------------------------------------------------------------------------------------------------------------------')

    # 如果误差更小，则保存模型
    if max_abs_error < maxerror:
        maxerror = max_abs_error
        torch.save(u.state_dict(), 'weights.pth')
        
    torch.save(u.state_dict(), 'weights11.pth')

    # 更新当前时间
    current_time = time.time()

    return maxerror, current_time


# 区域训练

## 区域1（0-0.6）g

In [16]:
def train1(change_weight, weight_pde, weight_JU, weight_boundary, epoch1, opt, u, maxerror, current_time, xy_values=None, u_values=None):
    
    for epoch in range(epoch1):
        opt.zero_grad()
    
        # 计算各个损失函数并获取点
        pde_loss, llos_pde, x_pde, y_pde = l_pde_1(u)
        JU_loss, llos_JU, x_JU, y_JU, p1, p2 = l_JU_1(u)
        ugeq_loss, llos_ugeq, x_ugeq, y_ugeq = l_ugeq_1(u)
    
        inside_loss = weight_pde * llos_pde + weight_JU * llos_JU + 6000 * llos_ugeq
        loss_boundary = l_boundary1(u) +  l_boundary2(u) + l_boundary3(u) + l_boundary4(u) + l_boundary5(u) + l_boundary6(u)
    
        # 计算加权总损失
        total_loss = weight_pde * pde_loss + weight_JU * JU_loss + 600 * ugeq_loss + weight_boundary * loss_boundary

        # 区域交换
        if xy_values is not None and u_values is not None:
            u_pred_boundary = u(xy_values)
            boundary_loss = torch.mean((u_pred_boundary - torch.tensor(u_values, device=xy_values.device))**2)
            total_loss += change_weight * boundary_loss
        # 每100次迭代记录一次损失
        if epoch % 100 == 0 and epoch != 0:
            print(f"Epoch {epoch}: pde_loss={pde_loss.item()}, JU_loss={JU_loss.item()}, boundary_loss={loss_boundary.item()}")
            # 记录 maxerror 并保存模型
            maxerror, current_time = record_loss_and_save_model(u, epoch, maxerror, current_time, h, device)

        # 反向传播和优化
        total_loss.backward()
        opt.step()
    
    print("\n训练完成，开始计算 x=0.6, y=0, 0.05, ..., 1 的 u 值")

    # **计算 x=-delta 的 u 值**
    y_values = torch.arange(0, 1.05, 0.05, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.02, 0.02, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.01, 0.01, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.005, 0.005, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.002, 0.002, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.001, 0.001, device=device).unsqueeze(1)
    
    x_values = torch.full_like(y_values, 0 - delta)  # 创建 x=0.4 的列向量
    xy_values = torch.cat([x_values, y_values], dim=1).to(device)

    # 用 u 计算这些点的值
    u_values = u(xy_values).detach().cpu().numpy()


    return xy_values, u_values, maxerror  


## 区域2（0.4-1）g

In [13]:
def train2(change_weight, weight_pde, weight_JU, weight_boundary, epoch2, opt, u, maxerror, current_time, xy_values=None, u_values=None):
    
    for epoch in range(epoch2):
        opt.zero_grad()
    
        # 计算各个损失函数并获取点
        pde_loss, llos_pde, x_pde, y_pde = l_pde_2(u)
        JU_loss, llos_JU, x_JU, y_JU, p1, p2 = l_JU_2(u)
        ugeq_loss, llos_ugeq, x_ugeq, y_ugeq = l_ugeq_2(u)
    
        inside_loss = weight_pde * llos_pde + weight_JU * llos_JU + 6000 * llos_ugeq
        loss_boundary = l_boundary1(u) +  l_boundary2(u) + l_boundary3(u) + l_boundary4(u) + l_boundary5(u) + l_boundary6(u)
    
        # 计算加权总损失
        total_loss = weight_pde * pde_loss + weight_JU * JU_loss + 600 * ugeq_loss + weight_boundary * loss_boundary
        
        # **如果传入了 xy_values 和 u_values，添加它们作为边界条件
        if xy_values is not None and u_values is not None:
            u_pred_boundary = u(xy_values)  # 计算神经网络的预测值
            boundary_loss = torch.mean((u_pred_boundary - torch.tensor(u_values, device=xy_values.device))**2)  # MSE损失
            total_loss += change_weight * boundary_loss  # 给边界损失较大权重
        # 每100次迭代记录一次损失
        if epoch % 100 == 0 and epoch != 0:
            print(f"pde:{pde_loss.item()},JU:{JU_loss.item()},boundary:{loss_boundary.item()}")
            maxerror, current_time = record_loss_and_save_model(u, epoch, maxerror, current_time, h, device)
        
        # if epoch % 100000 == 0 and epoch != 0:
        #     X, Y = update_dataset(X, Y, inside_loss, x_pde, y_pde, top_k, num_new_points, bias, device)
    
        # 反向传播和优化
        total_loss.backward()
        opt.step()
    
    print("\n训练完成，开始计算 x=0.4, y=0, 0.05, ..., 1 的 u 值")

    # **计算 x=+delta 的 u 值**
    y_values = torch.arange(0, 1.05, 0.05, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.02, 0.02, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.01, 0.01, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.005, 0.005, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.002, 0.002, device=device).unsqueeze(1)
    # y_values = torch.arange(0, 1.001, 0.001, device=device).unsqueeze(1)
    
    x_values = torch.full_like(y_values, 0 + delta)  # 创建 x=+delta 的列向量
    xy_values = torch.cat([x_values, y_values], dim=1).to(device)

    # **用 u 计算这些点的值**
    u_values = u(xy_values).detach().cpu().numpy()


    return xy_values, u_values, maxerror
    
    

# 训练

In [21]:
# Training
u = MLP().to(device)
opt = torch.optim.Adam(params=u.parameters(), lr=0.0001)

# 设置初始权重
weight_pde = 6000000000
weight_JU = 2000000
weight_boundary = 1000000000000
change_weight = 2000000
current_time = time.time()
start_time = time.time()
maxerror = 99999999999
epoch2 = epoch1 = 1001
steps = 2000
delta = 0.05

for step in range(steps):
    print(f"start to the {step} steps:")
    xy_values, u_values, maxerror = train1(change_weight, weight_pde, weight_JU, weight_boundary, epoch1, opt, u, maxerror, current_time)
    # if maxerror < 0.001:
    #     end_time = time.time()
    #     print(f"迭代停止步数为：{step+1},时间为：{end_time-start_time}")
    #     break
    print("*************************************************************train domain 1 finish, start to train domain 2*************************************************************")
    xy_values, u_values, maxerror = train2(change_weight, weight_pde, weight_JU, weight_boundary, epoch2, opt, u, maxerror, current_time)
    # if maxerror < 0.001:
    #     end_time = time.time()
    #     print(f"迭代停止步数为：{step+1},时间为：{end_time-start_time}")
    #     break
    print("*************************************************************train domain 2 finish, start to train domain 1*************************************************************")

print("                                                                        END TRAINING")

start to the 0 steps:
Epoch 100: pde_loss=114.26171112060547, JU_loss=-0.04192660003900528, boundary_loss=0.00030400531250052154
At epoch 100, time_speed: 2.39s, Max abs error is: 0.9997482299804688, best: 99999999999
-----------------------------------------------------------------------------------------------------------------------
Epoch 200: pde_loss=75.8173599243164, JU_loss=-0.053592558950185776, boundary_loss=0.00043227578862570226
At epoch 200, time_speed: 1.81s, Max abs error is: 0.9997482299804688, best: 0.9997482299804688
-----------------------------------------------------------------------------------------------------------------------
Epoch 300: pde_loss=82.4888687133789, JU_loss=-0.0838087871670723, boundary_loss=0.0007797694415785372
At epoch 300, time_speed: 1.96s, Max abs error is: 0.9997482299804688, best: 0.9997482299804688
-----------------------------------------------------------------------------------------------------------------------
Epoch 400: pde_loss=8

KeyboardInterrupt: 